# Optimal Tradin Rules - Numerical Determination

In [20]:
import numpy as np
import pandas as pd
from IPython.display import display, Latex, Markdown
import yfinance as yf

def print_md(x):
    display(Markdown(str(x)))

def fecho_tr(lista):
	return np.concatenate(lista, axis=None).tolist()

In [5]:
# # download data from yahoo finance
# data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

[*********************100%***********************]  2 of 2 completed


In [9]:
# aapl = data['Adj Close']['AAPL']

In [26]:
T = 12
I = 12

P = {i:{t: np.random.normal(0,1) for t in range(int(T))} for i in range(int(I))}

In [27]:
E_0P = {i: {t: np.random.normal(0,1) for t in range(int(T))} for i in range(int(I))}

In [28]:
T_ind = {i: i for i in range(int(I))}

## Step 1
We estimate the input parameters $\{\sigma, \varphi\}$ as:
$$
\underbrace{P_{i, t}}_{\mathbf Y}=\underbrace{\mathrm{E}_0\left[P_{i, T_i}\right]}_{\mathbf Z}+\varphi\underbrace{\left(P_{i, t-1}-\mathrm{E}_0\left[P_{i, T_i}\right]\right)}_{\mathbf X}+\xi_t
$$


In [44]:
X = pd.Series(fecho_tr([
    [P[i][t] - E_0P[i][T_ind[i]] 
     for t in range(int(T))] 
     for i in range(int(I))]
))

In [45]:
Y = pd.Series(fecho_tr([list(x.values()) for x in list(P.values())]))
Z = pd.Series(fecho_tr([list(x.values()) for x in list(E_0P.values())]))

Apply the OLS method to estimate $\sigma$ and $\varphi$:

In [48]:
from sklearn.linear_model import LinearRegression


In [74]:
# X_feat = pd.DataFrame({'X':X})
# y = Y-Z

# reg = LinearRegression(fit_intercept = False, penalty=None).fit(X_feat, y)

In [61]:
phi_hat = reg.coef_[0]

In [69]:
norm = lambda x: np.sqrt(np.sum(pd.to_numeric(x)**2))

cos = lambda x,y : np.dot(x,y)/(norm(x)*norm(y))

cov = lambda x,y : np.dot(x,y)/norm(y)

In [77]:
phi_hat = cov(Y,X)/cov(X,X)
xi_hat = Y-Z-phi_hat*X
sigma_hat = np.sqrt(cov(xi_hat, xi_hat))

In [79]:
print_md(f'''
$$\\hat\\varphi = {phi_hat} \\quad \\quad
\\hat\\sigma = {sigma_hat}$$
''')


$$\hat\varphi = 0.5922522943471566 \quad \quad
\hat\sigma = 3.802569027327373$$


# Step 2
We construct a mesh of stop-loss and profit-taking pairs, $(\underline{\pi}, \bar{\pi})$. For example, a Cartesian product of $\underline{\pi}=\left\{-\frac{1}{2} \sigma,-\sigma, \ldots,-10 \sigma\right\}$ and $\bar{\pi}=$ $\left\{\frac{1}{2} \sigma, \sigma, \ldots, 10 \sigma\right\}$ give us $20 \times 20$ nodes, each constituting an alternative trading rule $R \in \Omega$.


In [80]:
# Define the stop-loss and profit-taking levels
stop_loss_levels = np.linspace(-0.5, -10, num=20) * sigma_hat
profit_taking_levels = np.linspace(0.5, 10, num=20) * sigma_hat

# Generate the mesh of stop-loss and profit-taking pairs
stop_loss_profit_taking_pairs = np.array(np.meshgrid(stop_loss_levels, profit_taking_levels)).T.reshape(-1, 2)

# Print the pairs
pd.DataFrame(stop_loss_profit_taking_pairs)

,0,1
0,-1.901285,1.901285
1,-1.901285,3.802569
2,-1.901285,5.703854
3,-1.901285,7.605138
4,-1.901285,9.506423
...,...,...
395,-38.025690,30.420552
396,-38.025690,32.321837
397,-38.025690,34.223121
398,-38.025690,36.124406
